Disable all I/O so we can see the performance of the libraries not bound by I/O

In [1]:
import sys

class NullIO:
    """A file-like object that discards all writes"""
    def write(self, *args, **kwargs):
        pass
    def flush(self, *args, **kwargs):
        pass
    def close(self, *args, **kwargs):
        pass
    def __enter__(self):
        return self
    def __exit__(self, *args):
        pass

# Replace stdout and stderr with null I/O
sys.stdout = NullIO()
sys.stderr = NullIO()

### Don’t integrate

In [2]:
import structlog
import logging

#### Testing structlog

In [3]:
struct_logger = structlog.get_logger()

In [4]:
%%timeit -r 3 -n 10000
struct_logger.info("Hello World!")

### Testing stdlib

In [5]:
stdlib_logger = logging.getLogger()

In [6]:
%%timeit -r 3 -n 10000
stdlib_logger.info("Hello World!")

### [Rendering within structlog](https://www.structlog.org/en/stable/standard-library.html#rendering-within-structlog)

<svg aria-roledescription="flowchart-v2" role="graphics-document document" viewBox="0 0 440.66015625 478" style="max-width: 440.66015625px;" class="flowchart" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="100%" id="mermaid-1763366956645"><style>#mermaid-1763366956645{font-family:"trebuchet ms",verdana,arial,sans-serif;font-size:16px;fill:#333;}#mermaid-1763366956645 .error-icon{fill:#552222;}#mermaid-1763366956645 .error-text{fill:#552222;stroke:#552222;}#mermaid-1763366956645 .edge-thickness-normal{stroke-width:1px;}#mermaid-1763366956645 .edge-thickness-thick{stroke-width:3.5px;}#mermaid-1763366956645 .edge-pattern-solid{stroke-dasharray:0;}#mermaid-1763366956645 .edge-thickness-invisible{stroke-width:0;fill:none;}#mermaid-1763366956645 .edge-pattern-dashed{stroke-dasharray:3;}#mermaid-1763366956645 .edge-pattern-dotted{stroke-dasharray:2;}#mermaid-1763366956645 .marker{fill:#333333;stroke:#333333;}#mermaid-1763366956645 .marker.cross{stroke:#333333;}#mermaid-1763366956645 svg{font-family:"trebuchet ms",verdana,arial,sans-serif;font-size:16px;}#mermaid-1763366956645 p{margin:0;}#mermaid-1763366956645 .label{font-family:"trebuchet ms",verdana,arial,sans-serif;color:#333;}#mermaid-1763366956645 .cluster-label text{fill:#333;}#mermaid-1763366956645 .cluster-label span{color:#333;}#mermaid-1763366956645 .cluster-label span p{background-color:transparent;}#mermaid-1763366956645 .label text,#mermaid-1763366956645 span{fill:#333;color:#333;}#mermaid-1763366956645 .node rect,#mermaid-1763366956645 .node circle,#mermaid-1763366956645 .node ellipse,#mermaid-1763366956645 .node polygon,#mermaid-1763366956645 .node path{fill:#ECECFF;stroke:#9370DB;stroke-width:1px;}#mermaid-1763366956645 .rough-node .label text,#mermaid-1763366956645 .node .label text{text-anchor:middle;}#mermaid-1763366956645 .node .katex path{fill:#000;stroke:#000;stroke-width:1px;}#mermaid-1763366956645 .node .label{text-align:center;}#mermaid-1763366956645 .node.clickable{cursor:pointer;}#mermaid-1763366956645 .arrowheadPath{fill:#333333;}#mermaid-1763366956645 .edgePath .path{stroke:#333333;stroke-width:2.0px;}#mermaid-1763366956645 .flowchart-link{stroke:#333333;fill:none;}#mermaid-1763366956645 .edgeLabel{background-color:rgba(232,232,232, 0.8);text-align:center;}#mermaid-1763366956645 .edgeLabel p{background-color:rgba(232,232,232, 0.8);}#mermaid-1763366956645 .edgeLabel rect{opacity:0.5;background-color:rgba(232,232,232, 0.8);fill:rgba(232,232,232, 0.8);}#mermaid-1763366956645 .labelBkg{background-color:rgba(232, 232, 232, 0.5);}#mermaid-1763366956645 .cluster rect{fill:#ffffde;stroke:#aaaa33;stroke-width:1px;}#mermaid-1763366956645 .cluster text{fill:#333;}#mermaid-1763366956645 .cluster span{color:#333;}#mermaid-1763366956645 div.mermaidTooltip{position:absolute;text-align:center;max-width:200px;padding:2px;font-family:"trebuchet ms",verdana,arial,sans-serif;font-size:12px;background:hsl(80, 100%, 96.2745098039%);border:1px solid #aaaa33;border-radius:2px;pointer-events:none;z-index:100;}#mermaid-1763366956645 .flowchartTitleText{text-anchor:middle;font-size:18px;fill:#333;}#mermaid-1763366956645 :root{--mermaid-font-family:"trebuchet ms",verdana,arial,sans-serif;}</style><g><marker orient="auto" markerHeight="8" markerWidth="8" markerUnits="userSpaceOnUse" refY="5" refX="5" viewBox="0 0 10 10" class="marker flowchart-v2" id="mermaid-1763366956645_flowchart-v2-pointEnd"><path style="stroke-width: 1; stroke-dasharray: 1, 0;" class="arrowMarkerPath" d="M 0 0 L 10 5 L 0 10 z"></path></marker><marker orient="auto" markerHeight="8" markerWidth="8" markerUnits="userSpaceOnUse" refY="5" refX="4.5" viewBox="0 0 10 10" class="marker flowchart-v2" id="mermaid-1763366956645_flowchart-v2-pointStart"><path style="stroke-width: 1; stroke-dasharray: 1, 0;" class="arrowMarkerPath" d="M 0 5 L 10 10 L 10 0 z"></path></marker><marker orient="auto" markerHeight="11" markerWidth="11" markerUnits="userSpaceOnUse" refY="5" refX="11" viewBox="0 0 10 10" class="marker flowchart-v2" id="mermaid-1763366956645_flowchart-v2-circleEnd"><circle style="stroke-width: 1; stroke-dasharray: 1, 0;" class="arrowMarkerPath" r="5" cy="5" cx="5"></circle></marker><marker orient="auto" markerHeight="11" markerWidth="11" markerUnits="userSpaceOnUse" refY="5" refX="-1" viewBox="0 0 10 10" class="marker flowchart-v2" id="mermaid-1763366956645_flowchart-v2-circleStart"><circle style="stroke-width: 1; stroke-dasharray: 1, 0;" class="arrowMarkerPath" r="5" cy="5" cx="5"></circle></marker><marker orient="auto" markerHeight="11" markerWidth="11" markerUnits="userSpaceOnUse" refY="5.2" refX="12" viewBox="0 0 11 11" class="marker cross flowchart-v2" id="mermaid-1763366956645_flowchart-v2-crossEnd"><path style="stroke-width: 2; stroke-dasharray: 1, 0;" class="arrowMarkerPath" d="M 1,1 l 9,9 M 10,1 l -9,9"></path></marker><marker orient="auto" markerHeight="11" markerWidth="11" markerUnits="userSpaceOnUse" refY="5.2" refX="-1" viewBox="0 0 11 11" class="marker cross flowchart-v2" id="mermaid-1763366956645_flowchart-v2-crossStart"><path style="stroke-width: 2; stroke-dasharray: 1, 0;" class="arrowMarkerPath" d="M 1,1 l 9,9 M 10,1 l -9,9"></path></marker><g class="root"><g class="clusters"></g><g class="edgePaths"><path marker-end="url(#mermaid-1763366956645_flowchart-v2-pointEnd)" style="" class="edge-thickness-normal edge-pattern-solid edge-thickness-normal edge-pattern-solid flowchart-link" id="L_User_structlog_0" d="M258.743,62L268.526,68.167C278.309,74.333,297.875,86.667,307.658,95.917C317.441,105.167,317.441,111.333,317.441,116.833C317.441,122.333,317.441,127.167,317.441,129.583L317.441,132"></path><path marker-end="url(#mermaid-1763366956645_flowchart-v2-pointEnd)" style="" class="edge-thickness-normal edge-pattern-solid edge-thickness-normal edge-pattern-solid flowchart-link" id="L_User_stdlib_1" d="M173.074,62L163.291,68.167C153.508,74.333,133.941,86.667,124.158,103.5C114.375,120.333,114.375,141.667,114.375,165C114.375,188.333,114.375,213.667,119.086,230.417C123.798,247.167,133.22,255.333,142.139,263.063C151.058,270.793,159.473,278.087,163.68,281.734L167.888,285.38"></path><path marker-end="url(#mermaid-1763366956645_flowchart-v2-pointEnd)" style="" class="edge-thickness-normal edge-pattern-solid edge-thickness-normal edge-pattern-solid flowchart-link" id="L_structlog_stdlib_2" d="M317.441,190L317.441,198.167C317.441,206.333,317.441,222.667,312.73,234.917C308.019,247.167,298.596,255.333,289.677,263.063C280.759,270.793,272.344,278.087,268.136,281.734L263.929,285.38"></path><path marker-end="url(#mermaid-1763366956645_flowchart-v2-pointEnd)" style="" class="edge-thickness-thick edge-pattern-solid edge-thickness-normal edge-pattern-solid flowchart-link" id="L_stdlib_Output_3" d="M215.908,366L215.908,370.167C215.908,374.333,215.908,382.667,215.908,388.917C215.908,395.167,215.908,399.333,215.908,402.833C215.908,406.333,215.908,409.167,215.908,410.583L215.908,412"></path></g><g class="edgeLabels"><g transform="translate(317.44140625, 99)" class="edgeLabel"><g transform="translate(-115.21875, -12)" class="label"><foreignObject height="24" width="230.4375"><div style="display: table; white-space: break-spaces; line-height: 1.5; max-width: 200px; text-align: center; width: 200px;" class="labelBkg" xmlns="http://www.w3.org/1999/xhtml"><span class="edgeLabel"><p>structlog.get_logger().info('foo')</p></span></div></foreignObject></g></g><g transform="translate(114.375, 163)" class="edgeLabel"><g transform="translate(-106.375, -12)" class="label"><foreignObject height="24" width="212.75"><div style="display: table; white-space: break-spaces; line-height: 1.5; max-width: 200px; text-align: center; width: 200px;" class="labelBkg" xmlns="http://www.w3.org/1999/xhtml"><span class="edgeLabel"><p>logging.getLogger().info('foo')</p></span></div></foreignObject></g></g><g transform="translate(317.44140625, 239)" class="edgeLabel"><g transform="translate(-100, -24)" class="label"><foreignObject height="48" width="200"><div style="display: table; white-space: break-spaces; line-height: 1.5; max-width: 200px; text-align: center; width: 200px;" class="labelBkg" xmlns="http://www.w3.org/1999/xhtml"><span class="edgeLabel"><p>logging.getLogger().info("{'event': 'foo'}")</p></span></div></foreignObject></g></g><g class="edgeLabel"><g transform="translate(0, 0)" class="label"><foreignObject height="0" width="0"><div style="display: table-cell; white-space: nowrap; line-height: 1.5; max-width: 200px; text-align: center;" class="labelBkg" xmlns="http://www.w3.org/1999/xhtml"><span class="edgeLabel"></span></div></foreignObject></g></g></g><g class="nodes"><g transform="translate(215.908203125, 35)" id="flowchart-User-0" class="node default"><rect height="54" width="91.796875" y="-27" x="-45.8984375" data-et="node" data-id="abc" style="" class="basic label-container"></rect><g transform="translate(-15.8984375, -12)" style="" class="label"><rect></rect><foreignObject height="24" width="31.796875"><div style="display: table-cell; white-space: nowrap; line-height: 1.5; max-width: 200px; text-align: center;" xmlns="http://www.w3.org/1999/xhtml"><span class="nodeLabel"><p>User</p></span></div></foreignObject></g></g><g transform="translate(317.44140625, 163)" id="flowchart-structlog-1" class="node default"><rect height="54" width="123.3828125" y="-27" x="-61.69140625" data-et="node" data-id="abc" style="" class="basic label-container"></rect><g transform="translate(-31.69140625, -12)" style="" class="label"><rect></rect><foreignObject height="24" width="63.3828125"><div style="display: table-cell; white-space: nowrap; line-height: 1.5; max-width: 200px; text-align: center;" xmlns="http://www.w3.org/1999/xhtml"><span class="nodeLabel"><p>structlog</p></span></div></foreignObject></g></g><g transform="translate(215.908203125, 327)" id="flowchart-stdlib-2" class="node default"><rect height="78" width="256.7734375" y="-39" x="-128.38671875" data-et="node" data-id="abc" style="" class="basic label-container"></rect><g transform="translate(-98.38671875, -24)" style="" class="label"><rect></rect><foreignObject height="48" width="196.7734375"><div style="display: table-cell; white-space: nowrap; line-height: 1.5; max-width: 200px; text-align: center;" xmlns="http://www.w3.org/1999/xhtml"><span class="nodeLabel"><p>Standard Library<br>e.g. logging.StreamHandler</p></span></div></foreignObject></g></g><g transform="translate(215.908203125, 443)" id="flowchart-Output-9" class="node default"><rect height="54" width="109.8671875" y="-27" x="-54.93359375" data-et="node" data-id="abc" style="" class="basic label-container"></rect><g transform="translate(-24.93359375, -12)" style="" class="label"><rect></rect><foreignObject height="24" width="49.8671875"><div style="display: table-cell; white-space: nowrap; line-height: 1.5; max-width: 200px; text-align: center;" xmlns="http://www.w3.org/1999/xhtml"><span class="nodeLabel"><p>Output</p></span></div></foreignObject></g></g></g></g></g></svg>

In [7]:
import structlog

structlog.configure(
    processors=[
        # If log level is too low, abort pipeline and throw away log entry.
        structlog.stdlib.filter_by_level,
        # Add the name of the logger to event dict.
        structlog.stdlib.add_logger_name,
        # Add log level to event dict.
        structlog.stdlib.add_log_level,
        # Perform %-style formatting.
        structlog.stdlib.PositionalArgumentsFormatter(),
        # Add a timestamp in ISO 8601 format.
        structlog.processors.TimeStamper(fmt="iso"),
        # If the "stack_info" key in the event dict is true, remove it and
        # render the current stack trace in the "stack" key.
        structlog.processors.StackInfoRenderer(),
        # If the "exc_info" key in the event dict is either true or a
        # sys.exc_info() tuple, remove "exc_info" and render the exception
        # with traceback into the "exception" key.
        structlog.processors.format_exc_info,
        # If some value is in bytes, decode it to a Unicode str.
        structlog.processors.UnicodeDecoder(),
        # Add callsite parameters.
        structlog.processors.CallsiteParameterAdder(
            {
                structlog.processors.CallsiteParameter.FILENAME,
                structlog.processors.CallsiteParameter.FUNC_NAME,
                structlog.processors.CallsiteParameter.LINENO,
            }
        ),
        # Render the final event dict as JSON.
        structlog.processors.JSONRenderer()
    ],
    # `wrapper_class` is the bound logger that you get back from
    # get_logger(). This one imitates the API of `logging.Logger`.
    wrapper_class=structlog.stdlib.BoundLogger,
    # `logger_factory` is used to create wrapped loggers that are used for
    # OUTPUT. This one returns a `logging.Logger`. The final value (a JSON
    # string) from the final processor (`JSONRenderer`) will be passed to
    # the method of the same name as that you've called on the bound logger.
    logger_factory=structlog.stdlib.LoggerFactory(),
    # Effectively freeze configuration after creating the first bound
    # logger.
    cache_logger_on_first_use=True,
)

In [8]:
import logging
import sys

logging.basicConfig(
    format="%(message)s",
    stream=sys.stdout,
    level=logging.INFO,
)

#### Testing structlog

In [9]:
struct_logger = structlog.get_logger()

In [10]:
%%timeit -r 3 -n 10000
struct_logger.info("Hello World!")

### Testing stdlib

In [11]:
stdlib_logger = logging.getLogger()

In [12]:
%%timeit -r 3 -n 10000
stdlib_logger.info("Hello World!")

### [Rendering using logging-based formatters](https://www.structlog.org/en/stable/standard-library.html#rendering-using-logging-based-formatters)

<svg aria-roledescription="flowchart-v2" role="graphics-document document" viewBox="0 0 443.0703125 502" style="max-width: 443.0703125px;" class="flowchart" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="100%" id="mermaid-1763366956648"><style>#mermaid-1763366956648{font-family:"trebuchet ms",verdana,arial,sans-serif;font-size:16px;fill:#333;}#mermaid-1763366956648 .error-icon{fill:#552222;}#mermaid-1763366956648 .error-text{fill:#552222;stroke:#552222;}#mermaid-1763366956648 .edge-thickness-normal{stroke-width:1px;}#mermaid-1763366956648 .edge-thickness-thick{stroke-width:3.5px;}#mermaid-1763366956648 .edge-pattern-solid{stroke-dasharray:0;}#mermaid-1763366956648 .edge-thickness-invisible{stroke-width:0;fill:none;}#mermaid-1763366956648 .edge-pattern-dashed{stroke-dasharray:3;}#mermaid-1763366956648 .edge-pattern-dotted{stroke-dasharray:2;}#mermaid-1763366956648 .marker{fill:#333333;stroke:#333333;}#mermaid-1763366956648 .marker.cross{stroke:#333333;}#mermaid-1763366956648 svg{font-family:"trebuchet ms",verdana,arial,sans-serif;font-size:16px;}#mermaid-1763366956648 p{margin:0;}#mermaid-1763366956648 .label{font-family:"trebuchet ms",verdana,arial,sans-serif;color:#333;}#mermaid-1763366956648 .cluster-label text{fill:#333;}#mermaid-1763366956648 .cluster-label span{color:#333;}#mermaid-1763366956648 .cluster-label span p{background-color:transparent;}#mermaid-1763366956648 .label text,#mermaid-1763366956648 span{fill:#333;color:#333;}#mermaid-1763366956648 .node rect,#mermaid-1763366956648 .node circle,#mermaid-1763366956648 .node ellipse,#mermaid-1763366956648 .node polygon,#mermaid-1763366956648 .node path{fill:#ECECFF;stroke:#9370DB;stroke-width:1px;}#mermaid-1763366956648 .rough-node .label text,#mermaid-1763366956648 .node .label text{text-anchor:middle;}#mermaid-1763366956648 .node .katex path{fill:#000;stroke:#000;stroke-width:1px;}#mermaid-1763366956648 .node .label{text-align:center;}#mermaid-1763366956648 .node.clickable{cursor:pointer;}#mermaid-1763366956648 .arrowheadPath{fill:#333333;}#mermaid-1763366956648 .edgePath .path{stroke:#333333;stroke-width:2.0px;}#mermaid-1763366956648 .flowchart-link{stroke:#333333;fill:none;}#mermaid-1763366956648 .edgeLabel{background-color:rgba(232,232,232, 0.8);text-align:center;}#mermaid-1763366956648 .edgeLabel p{background-color:rgba(232,232,232, 0.8);}#mermaid-1763366956648 .edgeLabel rect{opacity:0.5;background-color:rgba(232,232,232, 0.8);fill:rgba(232,232,232, 0.8);}#mermaid-1763366956648 .labelBkg{background-color:rgba(232, 232, 232, 0.5);}#mermaid-1763366956648 .cluster rect{fill:#ffffde;stroke:#aaaa33;stroke-width:1px;}#mermaid-1763366956648 .cluster text{fill:#333;}#mermaid-1763366956648 .cluster span{color:#333;}#mermaid-1763366956648 div.mermaidTooltip{position:absolute;text-align:center;max-width:200px;padding:2px;font-family:"trebuchet ms",verdana,arial,sans-serif;font-size:12px;background:hsl(80, 100%, 96.2745098039%);border:1px solid #aaaa33;border-radius:2px;pointer-events:none;z-index:100;}#mermaid-1763366956648 .flowchartTitleText{text-anchor:middle;font-size:18px;fill:#333;}#mermaid-1763366956648 :root{--mermaid-font-family:"trebuchet ms",verdana,arial,sans-serif;}</style><g><marker orient="auto" markerHeight="8" markerWidth="8" markerUnits="userSpaceOnUse" refY="5" refX="5" viewBox="0 0 10 10" class="marker flowchart-v2" id="mermaid-1763366956648_flowchart-v2-pointEnd"><path style="stroke-width: 1; stroke-dasharray: 1, 0;" class="arrowMarkerPath" d="M 0 0 L 10 5 L 0 10 z"></path></marker><marker orient="auto" markerHeight="8" markerWidth="8" markerUnits="userSpaceOnUse" refY="5" refX="4.5" viewBox="0 0 10 10" class="marker flowchart-v2" id="mermaid-1763366956648_flowchart-v2-pointStart"><path style="stroke-width: 1; stroke-dasharray: 1, 0;" class="arrowMarkerPath" d="M 0 5 L 10 10 L 10 0 z"></path></marker><marker orient="auto" markerHeight="11" markerWidth="11" markerUnits="userSpaceOnUse" refY="5" refX="11" viewBox="0 0 10 10" class="marker flowchart-v2" id="mermaid-1763366956648_flowchart-v2-circleEnd"><circle style="stroke-width: 1; stroke-dasharray: 1, 0;" class="arrowMarkerPath" r="5" cy="5" cx="5"></circle></marker><marker orient="auto" markerHeight="11" markerWidth="11" markerUnits="userSpaceOnUse" refY="5" refX="-1" viewBox="0 0 10 10" class="marker flowchart-v2" id="mermaid-1763366956648_flowchart-v2-circleStart"><circle style="stroke-width: 1; stroke-dasharray: 1, 0;" class="arrowMarkerPath" r="5" cy="5" cx="5"></circle></marker><marker orient="auto" markerHeight="11" markerWidth="11" markerUnits="userSpaceOnUse" refY="5.2" refX="12" viewBox="0 0 11 11" class="marker cross flowchart-v2" id="mermaid-1763366956648_flowchart-v2-crossEnd"><path style="stroke-width: 2; stroke-dasharray: 1, 0;" class="arrowMarkerPath" d="M 1,1 l 9,9 M 10,1 l -9,9"></path></marker><marker orient="auto" markerHeight="11" markerWidth="11" markerUnits="userSpaceOnUse" refY="5.2" refX="-1" viewBox="0 0 11 11" class="marker cross flowchart-v2" id="mermaid-1763366956648_flowchart-v2-crossStart"><path style="stroke-width: 2; stroke-dasharray: 1, 0;" class="arrowMarkerPath" d="M 1,1 l 9,9 M 10,1 l -9,9"></path></marker><g class="root"><g class="clusters"></g><g class="edgePaths"><path marker-end="url(#mermaid-1763366956648_flowchart-v2-pointEnd)" style="" class="edge-thickness-normal edge-pattern-solid edge-thickness-normal edge-pattern-solid flowchart-link" id="L_User_structlog_0" d="M251.979,62L262.89,70.167C273.8,78.333,295.621,94.667,306.531,106.917C317.441,119.167,317.441,127.333,317.441,134.833C317.441,142.333,317.441,149.167,317.441,152.583L317.441,156"></path><path marker-end="url(#mermaid-1763366956648_flowchart-v2-pointEnd)" style="" class="edge-thickness-normal edge-pattern-solid edge-thickness-normal edge-pattern-solid flowchart-link" id="L_User_stdlib_1" d="M179.837,62L168.927,70.167C158.016,78.333,136.196,94.667,125.285,115.5C114.375,136.333,114.375,161.667,114.375,187C114.375,212.333,114.375,237.667,119.086,254.417C123.798,271.167,133.22,279.333,142.139,287.063C151.058,294.793,159.473,302.087,163.68,305.734L167.888,309.38"></path><path marker-end="url(#mermaid-1763366956648_flowchart-v2-pointEnd)" style="" class="edge-thickness-normal edge-pattern-solid edge-thickness-normal edge-pattern-solid flowchart-link" id="L_structlog_stdlib_2" d="M317.441,214L317.441,222.167C317.441,230.333,317.441,246.667,312.73,258.917C308.019,271.167,298.596,279.333,289.677,287.063C280.759,294.793,272.344,302.087,268.136,305.734L263.929,309.38"></path><path marker-end="url(#mermaid-1763366956648_flowchart-v2-pointEnd)" style="" class="edge-thickness-thick edge-pattern-solid edge-thickness-normal edge-pattern-solid flowchart-link" id="L_stdlib_Output_3" d="M215.908,390L215.908,394.167C215.908,398.333,215.908,406.667,215.908,412.917C215.908,419.167,215.908,423.333,215.908,426.833C215.908,430.333,215.908,433.167,215.908,434.583L215.908,436"></path></g><g class="edgeLabels"><g transform="translate(317.44140625, 111)" class="edgeLabel"><g transform="translate(-117.62890625, -24)" class="label"><foreignObject height="48" width="235.2578125"><div style="display: table; white-space: break-spaces; line-height: 1.5; max-width: 200px; text-align: center; width: 200px;" class="labelBkg" xmlns="http://www.w3.org/1999/xhtml"><span class="edgeLabel"><p>structlog.get_logger().info('foo', bar=42)</p></span></div></foreignObject></g></g><g transform="translate(114.375, 187)" class="edgeLabel"><g transform="translate(-106.375, -12)" class="label"><foreignObject height="24" width="212.75"><div style="display: table; white-space: break-spaces; line-height: 1.5; max-width: 200px; text-align: center; width: 200px;" class="labelBkg" xmlns="http://www.w3.org/1999/xhtml"><span class="edgeLabel"><p>logging.getLogger().info('foo')</p></span></div></foreignObject></g></g><g transform="translate(317.44140625, 263)" class="edgeLabel"><g transform="translate(-108.78515625, -24)" class="label"><foreignObject height="48" width="217.5703125"><div style="display: table; white-space: break-spaces; line-height: 1.5; max-width: 200px; text-align: center; width: 200px;" class="labelBkg" xmlns="http://www.w3.org/1999/xhtml"><span class="edgeLabel"><p>logging.getLogger().info('foo', extra={"bar": 42})</p></span></div></foreignObject></g></g><g class="edgeLabel"><g transform="translate(0, 0)" class="label"><foreignObject height="0" width="0"><div style="display: table-cell; white-space: nowrap; line-height: 1.5; max-width: 200px; text-align: center;" class="labelBkg" xmlns="http://www.w3.org/1999/xhtml"><span class="edgeLabel"></span></div></foreignObject></g></g></g><g class="nodes"><g transform="translate(215.908203125, 35)" id="flowchart-User-11" class="node default"><rect height="54" width="91.796875" y="-27" x="-45.8984375" data-et="node" data-id="abc" style="" class="basic label-container"></rect><g transform="translate(-15.8984375, -12)" style="" class="label"><rect></rect><foreignObject height="24" width="31.796875"><div style="display: table-cell; white-space: nowrap; line-height: 1.5; max-width: 200px; text-align: center;" xmlns="http://www.w3.org/1999/xhtml"><span class="nodeLabel"><p>User</p></span></div></foreignObject></g></g><g transform="translate(317.44140625, 187)" id="flowchart-structlog-12" class="node default"><rect height="54" width="123.3828125" y="-27" x="-61.69140625" data-et="node" data-id="abc" style="" class="basic label-container"></rect><g transform="translate(-31.69140625, -12)" style="" class="label"><rect></rect><foreignObject height="24" width="63.3828125"><div style="display: table-cell; white-space: nowrap; line-height: 1.5; max-width: 200px; text-align: center;" xmlns="http://www.w3.org/1999/xhtml"><span class="nodeLabel"><p>structlog</p></span></div></foreignObject></g></g><g transform="translate(215.908203125, 351)" id="flowchart-stdlib-13" class="node default"><rect height="78" width="256.7734375" y="-39" x="-128.38671875" data-et="node" data-id="abc" style="" class="basic label-container"></rect><g transform="translate(-98.38671875, -24)" style="" class="label"><rect></rect><foreignObject height="48" width="196.7734375"><div style="display: table-cell; white-space: nowrap; line-height: 1.5; max-width: 200px; text-align: center;" xmlns="http://www.w3.org/1999/xhtml"><span class="nodeLabel"><p>Standard Library<br>e.g. logging.StreamHandler</p></span></div></foreignObject></g></g><g transform="translate(215.908203125, 467)" id="flowchart-Output-20" class="node default"><rect height="54" width="109.8671875" y="-27" x="-54.93359375" data-et="node" data-id="abc" style="" class="basic label-container"></rect><g transform="translate(-24.93359375, -12)" style="" class="label"><rect></rect><foreignObject height="24" width="49.8671875"><div style="display: table-cell; white-space: nowrap; line-height: 1.5; max-width: 200px; text-align: center;" xmlns="http://www.w3.org/1999/xhtml"><span class="nodeLabel"><p>Output</p></span></div></foreignObject></g></g></g></g></g></svg>


In [13]:
import structlog

structlog.configure(
    processors=[
        structlog.stdlib.filter_by_level,
        structlog.stdlib.add_logger_name,
        structlog.stdlib.add_log_level,
        structlog.stdlib.PositionalArgumentsFormatter(),
        structlog.processors.StackInfoRenderer(),
        structlog.processors.format_exc_info,
        structlog.processors.UnicodeDecoder(),
        # Transform event dict into `logging.Logger` method arguments.
        # "event" becomes "msg" and the rest is passed as a dict in
        # "extra". IMPORTANT: This means that the standard library MUST
        # render "extra" for the context to appear in log entries! See
        # warning below.
        structlog.stdlib.render_to_log_kwargs,
    ],
    logger_factory=structlog.stdlib.LoggerFactory(),
    wrapper_class=structlog.stdlib.BoundLogger,
    cache_logger_on_first_use=True,
)

In [14]:
import logging
import sys

from pythonjsonlogger import jsonlogger

handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(jsonlogger.JsonFormatter())
root_logger = logging.getLogger()
root_logger.addHandler(handler)

#### Testing structlog

In [15]:
struct_logger = structlog.get_logger()

In [16]:
%%timeit -r 3 -n 10000
struct_logger.info("Hello World!")

### Testing stdlib

In [17]:
stdlib_logger = logging.getLogger()

In [18]:
%%timeit -r 3 -n 10000
stdlib_logger.info("Hello World!")

### [Rendering using structlog-based formatters within logging](https://www.structlog.org/en/stable/standard-library.html#rendering-using-structlog-based-formatters-within-logging)

<svg aria-roledescription="flowchart-v2" role="graphics-document document" viewBox="0 0 464.205078125 878" style="max-width: 464.205078125px;" class="flowchart" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="100%" id="mermaid-1763366957071"><style>#mermaid-1763366957071{font-family:"trebuchet ms",verdana,arial,sans-serif;font-size:16px;fill:#333;}#mermaid-1763366957071 .error-icon{fill:#552222;}#mermaid-1763366957071 .error-text{fill:#552222;stroke:#552222;}#mermaid-1763366957071 .edge-thickness-normal{stroke-width:1px;}#mermaid-1763366957071 .edge-thickness-thick{stroke-width:3.5px;}#mermaid-1763366957071 .edge-pattern-solid{stroke-dasharray:0;}#mermaid-1763366957071 .edge-thickness-invisible{stroke-width:0;fill:none;}#mermaid-1763366957071 .edge-pattern-dashed{stroke-dasharray:3;}#mermaid-1763366957071 .edge-pattern-dotted{stroke-dasharray:2;}#mermaid-1763366957071 .marker{fill:#333333;stroke:#333333;}#mermaid-1763366957071 .marker.cross{stroke:#333333;}#mermaid-1763366957071 svg{font-family:"trebuchet ms",verdana,arial,sans-serif;font-size:16px;}#mermaid-1763366957071 p{margin:0;}#mermaid-1763366957071 .label{font-family:"trebuchet ms",verdana,arial,sans-serif;color:#333;}#mermaid-1763366957071 .cluster-label text{fill:#333;}#mermaid-1763366957071 .cluster-label span{color:#333;}#mermaid-1763366957071 .cluster-label span p{background-color:transparent;}#mermaid-1763366957071 .label text,#mermaid-1763366957071 span{fill:#333;color:#333;}#mermaid-1763366957071 .node rect,#mermaid-1763366957071 .node circle,#mermaid-1763366957071 .node ellipse,#mermaid-1763366957071 .node polygon,#mermaid-1763366957071 .node path{fill:#ECECFF;stroke:#9370DB;stroke-width:1px;}#mermaid-1763366957071 .rough-node .label text,#mermaid-1763366957071 .node .label text{text-anchor:middle;}#mermaid-1763366957071 .node .katex path{fill:#000;stroke:#000;stroke-width:1px;}#mermaid-1763366957071 .node .label{text-align:center;}#mermaid-1763366957071 .node.clickable{cursor:pointer;}#mermaid-1763366957071 .arrowheadPath{fill:#333333;}#mermaid-1763366957071 .edgePath .path{stroke:#333333;stroke-width:2.0px;}#mermaid-1763366957071 .flowchart-link{stroke:#333333;fill:none;}#mermaid-1763366957071 .edgeLabel{background-color:rgba(232,232,232, 0.8);text-align:center;}#mermaid-1763366957071 .edgeLabel p{background-color:rgba(232,232,232, 0.8);}#mermaid-1763366957071 .edgeLabel rect{opacity:0.5;background-color:rgba(232,232,232, 0.8);fill:rgba(232,232,232, 0.8);}#mermaid-1763366957071 .labelBkg{background-color:rgba(232, 232, 232, 0.5);}#mermaid-1763366957071 .cluster rect{fill:#ffffde;stroke:#aaaa33;stroke-width:1px;}#mermaid-1763366957071 .cluster text{fill:#333;}#mermaid-1763366957071 .cluster span{color:#333;}#mermaid-1763366957071 div.mermaidTooltip{position:absolute;text-align:center;max-width:200px;padding:2px;font-family:"trebuchet ms",verdana,arial,sans-serif;font-size:12px;background:hsl(80, 100%, 96.2745098039%);border:1px solid #aaaa33;border-radius:2px;pointer-events:none;z-index:100;}#mermaid-1763366957071 .flowchartTitleText{text-anchor:middle;font-size:18px;fill:#333;}#mermaid-1763366957071 :root{--mermaid-font-family:"trebuchet ms",verdana,arial,sans-serif;}</style><g><marker orient="auto" markerHeight="8" markerWidth="8" markerUnits="userSpaceOnUse" refY="5" refX="5" viewBox="0 0 10 10" class="marker flowchart-v2" id="mermaid-1763366957071_flowchart-v2-pointEnd"><path style="stroke-width: 1; stroke-dasharray: 1, 0;" class="arrowMarkerPath" d="M 0 0 L 10 5 L 0 10 z"></path></marker><marker orient="auto" markerHeight="8" markerWidth="8" markerUnits="userSpaceOnUse" refY="5" refX="4.5" viewBox="0 0 10 10" class="marker flowchart-v2" id="mermaid-1763366957071_flowchart-v2-pointStart"><path style="stroke-width: 1; stroke-dasharray: 1, 0;" class="arrowMarkerPath" d="M 0 5 L 10 10 L 10 0 z"></path></marker><marker orient="auto" markerHeight="11" markerWidth="11" markerUnits="userSpaceOnUse" refY="5" refX="11" viewBox="0 0 10 10" class="marker flowchart-v2" id="mermaid-1763366957071_flowchart-v2-circleEnd"><circle style="stroke-width: 1; stroke-dasharray: 1, 0;" class="arrowMarkerPath" r="5" cy="5" cx="5"></circle></marker><marker orient="auto" markerHeight="11" markerWidth="11" markerUnits="userSpaceOnUse" refY="5" refX="-1" viewBox="0 0 10 10" class="marker flowchart-v2" id="mermaid-1763366957071_flowchart-v2-circleStart"><circle style="stroke-width: 1; stroke-dasharray: 1, 0;" class="arrowMarkerPath" r="5" cy="5" cx="5"></circle></marker><marker orient="auto" markerHeight="11" markerWidth="11" markerUnits="userSpaceOnUse" refY="5.2" refX="12" viewBox="0 0 11 11" class="marker cross flowchart-v2" id="mermaid-1763366957071_flowchart-v2-crossEnd"><path style="stroke-width: 2; stroke-dasharray: 1, 0;" class="arrowMarkerPath" d="M 1,1 l 9,9 M 10,1 l -9,9"></path></marker><marker orient="auto" markerHeight="11" markerWidth="11" markerUnits="userSpaceOnUse" refY="5.2" refX="-1" viewBox="0 0 11 11" class="marker cross flowchart-v2" id="mermaid-1763366957071_flowchart-v2-crossStart"><path style="stroke-width: 2; stroke-dasharray: 1, 0;" class="arrowMarkerPath" d="M 1,1 l 9,9 M 10,1 l -9,9"></path></marker><g class="root"><g class="clusters"></g><g class="edgePaths"><path marker-end="url(#mermaid-1763366957071_flowchart-v2-pointEnd)" style="" class="edge-thickness-normal edge-pattern-solid edge-thickness-normal edge-pattern-solid flowchart-link" id="L_User_structlog_0" d="M256.95,62L268.002,70.167C279.055,78.333,301.159,94.667,312.211,106.917C323.264,119.167,323.264,127.333,323.264,134.833C323.264,142.333,323.264,149.167,323.264,152.583L323.264,156"></path><path marker-end="url(#mermaid-1763366957071_flowchart-v2-pointEnd)" style="" class="edge-thickness-normal edge-pattern-solid edge-thickness-normal edge-pattern-solid flowchart-link" id="L_User_stdlib_1" d="M183.87,62L172.818,70.167C161.766,78.333,139.661,94.667,128.609,115.5C117.557,136.333,117.557,161.667,117.557,189C117.557,216.333,117.557,245.667,123.498,265.417C129.439,285.167,141.322,295.333,152.698,305.067C164.074,314.8,174.944,324.1,180.379,328.75L185.813,333.4"></path><path marker-end="url(#mermaid-1763366957071_flowchart-v2-pointEnd)" style="" class="edge-thickness-normal edge-pattern-solid edge-thickness-normal edge-pattern-solid flowchart-link" id="L_structlog_stdlib_2" d="M323.264,214L323.264,224.167C323.264,234.333,323.264,254.667,317.322,269.917C311.381,285.167,299.498,295.333,288.122,305.067C276.746,314.8,265.876,324.1,260.442,328.75L255.007,333.4"></path><path marker-end="url(#mermaid-1763366957071_flowchart-v2-pointEnd)" style="" class="edge-thickness-normal edge-pattern-solid edge-thickness-normal edge-pattern-solid flowchart-link" id="L_stdlib_structlog2_3" d="M220.41,390L220.41,396.167C220.41,402.333,220.41,414.667,220.41,423.917C220.41,433.167,220.41,439.333,220.41,444.833C220.41,450.333,220.41,455.167,220.41,457.583L220.41,460"></path><path marker-end="url(#mermaid-1763366957071_flowchart-v2-pointEnd)" style="" class="edge-thickness-normal edge-pattern-solid edge-thickness-normal edge-pattern-solid flowchart-link" id="L_structlog2_stdlib2_4" d="M220.41,518L220.41,532.167C220.41,546.333,220.41,574.667,220.41,595.917C220.41,617.167,220.41,631.333,220.41,644.833C220.41,658.333,220.41,671.167,220.41,677.583L220.41,684"></path><path marker-end="url(#mermaid-1763366957071_flowchart-v2-pointEnd)" style="" class="edge-thickness-thick edge-pattern-solid edge-thickness-normal edge-pattern-solid flowchart-link" id="L_stdlib2_Output_5" d="M220.41,766L220.41,770.167C220.41,774.333,220.41,782.667,220.41,788.917C220.41,795.167,220.41,799.333,220.41,802.833C220.41,806.333,220.41,809.167,220.41,810.583L220.41,812"></path></g><g class="edgeLabels"><g transform="translate(323.263671875, 111)" class="edgeLabel"><g transform="translate(-120.26953125, -24)" class="label"><foreignObject height="48" width="240.5390625"><div style="display: table; white-space: break-spaces; line-height: 1.5; max-width: 200px; text-align: center; width: 200px;" class="labelBkg" xmlns="http://www.w3.org/1999/xhtml"><span class="edgeLabel"><p>structlog.get_logger().info("foo", bar=42)</p></span></div></foreignObject></g></g><g transform="translate(117.556640625, 187)" class="edgeLabel"><g transform="translate(-109.015625, -12)" class="label"><foreignObject height="24" width="218.03125"><div style="display: table; white-space: break-spaces; line-height: 1.5; max-width: 200px; text-align: center; width: 200px;" class="labelBkg" xmlns="http://www.w3.org/1999/xhtml"><span class="edgeLabel"><p>logging.getLogger().info("foo")</p></span></div></foreignObject></g></g><g transform="translate(323.263671875, 275)" class="edgeLabel"><g transform="translate(-132.94140625, -36)" class="label"><foreignObject height="72" width="265.8828125"><div style="display: table; white-space: break-spaces; line-height: 1.5; max-width: 200px; text-align: center; width: 200px;" class="labelBkg" xmlns="http://www.w3.org/1999/xhtml"><span class="edgeLabel"><p>logging.getLogger().info(event_dict, {"extra": {"_logger": logger, "_name": name})</p></span></div></foreignObject></g></g><g transform="translate(220.41015625, 427)" class="edgeLabel"><g transform="translate(-212.41015625, -12)" class="label"><foreignObject height="24" width="424.8203125"><div style="display: table; white-space: break-spaces; line-height: 1.5; max-width: 200px; text-align: center; width: 200px;" class="labelBkg" xmlns="http://www.w3.org/1999/xhtml"><span class="edgeLabel"><p>structlog.stdlib.ProcessorFormatter.format(logging.Record)</p></span></div></foreignObject></g></g><g transform="translate(220.41015625, 603)" class="edgeLabel"><g transform="translate(-100, -60)" class="label"><foreignObject height="120" width="200"><div style="display: table; white-space: break-spaces; line-height: 1.5; max-width: 200px; text-align: center; width: 200px;" class="labelBkg" xmlns="http://www.w3.org/1999/xhtml"><span class="edgeLabel"><p>Returns a string that is passed into logging handlers.<br>This flow is controlled by the logging configuration.</p></span></div></foreignObject></g></g><g class="edgeLabel"><g transform="translate(0, 0)" class="label"><foreignObject height="0" width="0"><div style="display: table-cell; white-space: nowrap; line-height: 1.5; max-width: 200px; text-align: center;" class="labelBkg" xmlns="http://www.w3.org/1999/xhtml"><span class="edgeLabel"></span></div></foreignObject></g></g></g><g class="nodes"><g transform="translate(220.41015625, 35)" id="flowchart-User-22" class="node default"><rect height="54" width="91.796875" y="-27" x="-45.8984375" data-et="node" data-id="abc" style="" class="basic label-container"></rect><g transform="translate(-15.8984375, -12)" style="" class="label"><rect></rect><foreignObject height="24" width="31.796875"><div style="display: table-cell; white-space: nowrap; line-height: 1.5; max-width: 200px; text-align: center;" xmlns="http://www.w3.org/1999/xhtml"><span class="nodeLabel"><p>User</p></span></div></foreignObject></g></g><g transform="translate(323.263671875, 187)" id="flowchart-structlog-23" class="node default"><rect height="54" width="123.3828125" y="-27" x="-61.69140625" data-et="node" data-id="abc" style="" class="basic label-container"></rect><g transform="translate(-31.69140625, -12)" style="" class="label"><rect></rect><foreignObject height="24" width="63.3828125"><div style="display: table-cell; white-space: nowrap; line-height: 1.5; max-width: 200px; text-align: center;" xmlns="http://www.w3.org/1999/xhtml"><span class="nodeLabel"><p>structlog</p></span></div></foreignObject></g></g><g transform="translate(220.41015625, 491)" id="flowchart-structlog2-24" class="node default"><rect height="54" width="123.3828125" y="-27" x="-61.69140625" data-et="node" data-id="abc" style="" class="basic label-container"></rect><g transform="translate(-31.69140625, -12)" style="" class="label"><rect></rect><foreignObject height="24" width="63.3828125"><div style="display: table-cell; white-space: nowrap; line-height: 1.5; max-width: 200px; text-align: center;" xmlns="http://www.w3.org/1999/xhtml"><span class="nodeLabel"><p>structlog</p></span></div></foreignObject></g></g><g transform="translate(220.41015625, 363)" id="flowchart-stdlib-25" class="node default"><rect height="54" width="178.7734375" y="-27" x="-89.38671875" data-et="node" data-id="abc" style="" class="basic label-container"></rect><g transform="translate(-59.38671875, -12)" style="" class="label"><rect></rect><foreignObject height="24" width="118.7734375"><div style="display: table-cell; white-space: nowrap; line-height: 1.5; max-width: 200px; text-align: center;" xmlns="http://www.w3.org/1999/xhtml"><span class="nodeLabel"><p>Standard Library</p></span></div></foreignObject></g></g><g transform="translate(220.41015625, 727)" id="flowchart-stdlib2-35" class="node default"><rect height="78" width="256.7734375" y="-39" x="-128.38671875" data-et="node" data-id="abc" style="" class="basic label-container"></rect><g transform="translate(-98.38671875, -24)" style="" class="label"><rect></rect><foreignObject height="48" width="196.7734375"><div style="display: table-cell; white-space: nowrap; line-height: 1.5; max-width: 200px; text-align: center;" xmlns="http://www.w3.org/1999/xhtml"><span class="nodeLabel"><p>Standard Library<br>e.g. logging.StreamHandler</p></span></div></foreignObject></g></g><g transform="translate(220.41015625, 843)" id="flowchart-Output-37" class="node default"><rect height="54" width="109.8671875" y="-27" x="-54.93359375" data-et="node" data-id="abc" style="" class="basic label-container"></rect><g transform="translate(-24.93359375, -12)" style="" class="label"><rect></rect><foreignObject height="24" width="49.8671875"><div style="display: table-cell; white-space: nowrap; line-height: 1.5; max-width: 200px; text-align: center;" xmlns="http://www.w3.org/1999/xhtml"><span class="nodeLabel"><p>Output</p></span></div></foreignObject></g></g></g></g></g></svg>

In [19]:
import logging
import structlog

structlog.configure(
    processors=[
        # Prepare event dict for `ProcessorFormatter`.
        structlog.stdlib.ProcessorFormatter.wrap_for_formatter,
    ],
    logger_factory=structlog.stdlib.LoggerFactory(),
)

formatter = structlog.stdlib.ProcessorFormatter(
    processors=[structlog.dev.ConsoleRenderer()],
)

handler = logging.StreamHandler()
# Use OUR `ProcessorFormatter` to format all `logging` entries.
handler.setFormatter(formatter)
root_logger = logging.getLogger()
root_logger.addHandler(handler)
root_logger.setLevel(logging.INFO)

#### Testing structlog

In [20]:
struct_logger = structlog.get_logger()

In [21]:
%%timeit -r 3 -n 10000
struct_logger.info("Hello World!")

### Testing stdlib

In [22]:
stdlib_logger = logging.getLogger()

In [23]:
%%timeit -r 3 -n 10000
stdlib_logger.info("Hello World!")